In [ ]:
!unzip "/content/drive/MyDrive/IDRID_MESSIDOR.zip"

Archive:  /content/drive/MyDrive/IDRID_MESSIDOR.zip
   creating: IDRID_MESSIDOR/Dataset/
   creating: IDRID_MESSIDOR/Dataset/TestSet/
   creating: IDRID_MESSIDOR/Dataset/TestSet/0/
  inflating: IDRID_MESSIDOR/Dataset/TestSet/0/IDRiD_001.jpg  
  inflating: IDRID_MESSIDOR/Dataset/TestSet/0/IDRiD_003.jpg  
  inflating: IDRID_MESSIDOR/Dataset/TestSet/0/IDRiD_005.jpg  
  inflating: IDRID_MESSIDOR/Dataset/TestSet/0/IDRiD_007.jpg  
  inflating: IDRID_MESSIDOR/Dataset/TestSet/0/IDRiD_029.jpg  
  inflating: IDRID_MESSIDOR/Dataset/TestSet/0/IDRiD_030.jpg  
  inflating: IDRID_MESSIDOR/Dataset/TestSet/0/IDRiD_031.jpg  
  inflating: IDRID_MESSIDOR/Dataset/TestSet/0/IDRiD_037.jpg  
  inflating: IDRID_MESSIDOR/Dataset/TestSet/0/IDRiD_038.jpg  
  inflating: IDRID_MESSIDOR/Dataset/TestSet/0/IDRiD_039.jpg  
  inflating: IDRID_MESSIDOR/Dataset/TestSet/0/IDRiD_041.jpg  
  inflating: IDRID_MESSIDOR/Dataset/TestSet/0/IDRiD_043.jpg  
  inflating: IDRID_MESSIDOR/Dataset/TestSet/0/IDRiD_044.jpg  
  inflating: 

In [ ]:
import cv2
import numpy as np
from numpy import std, mean, sqrt, max, min, exp
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import InceptionResNetV2

import matplotlib.pyplot as plt
import os
import sklearn
from sklearn.utils import shuffle
from sklearn import metrics
from sklearn.metrics import confusion_matrix,precision_score,recall_score,accuracy_score
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
print("Done Importing")

In [ ]:
def normalize(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))
# Color Transormation
def ColorTransition(Simg, Timg):
    R, G, B=cv2.split(Simg)
    R = np.float64(R)
    G = np.float64(G)
    B = np.float64(B)
 
    R1, G1, B1=cv2.split(Timg)
    R1 = np.float64(R1)
    G1 = np.float64(G1)
    B1 = np.float64(B1)
 
    # conversion from RGB to lab color space -> Source Image
    L=0.3811*R+0.5783*G+0.0402*B
    M=0.1967*R+0.7244*G+0.0782*B
    S=0.0241*R+0.1288*G+0.8444*B
    L = np.float64(L)
    M = np.float64(M)
    S = np.float64(S)
 
    # conversion from RGB to lab color space -> Target Image
    L1=0.3811*R1+0.5783*G1+0.0402*B1;
    M1=0.1967*R1+0.7244*G1+0.0782*B1;
    S1=0.0241*R1+0.1288*G1+0.8444*B1;
    L1 = np.float64(L1)
    M1 = np.float64(M1)
    S1 = np.float64(S1)
 
    I2 = cv2.merge((L,M,S))
    A2 = cv2.merge((L1,M1,S1))
    
    l=0.5774*L+0.5774*M+0.5774*S;
    a=0.4082*L+0.4082*M-0.8165*S;
    b=0.7071*L-0.7071*M;
    l = np.float64(l)
    a = np.float64(a)
    b = np.float64(b)
 
    l1=0.5774*L1+0.5774*M1+0.5774*S1;
    a1=0.4082*L1+0.4082*M1-0.8165*S1;
    b1=0.7071*L1-0.7071*M1;
    l1 = np.float64(l1)
    a1 = np.float64(a1)
    b1 = np.float64(b1)
 
    I3 = cv2.merge((l,a,b))
    A3 = cv2.merge((l1,a1,b1))
 
    std1=std(l1)
    std2=std(l)
 
    std3=std(a1);
    std4=std(a);
    
    std5=std(b1);
    std6=std(b);
 
    p=(sqrt(mean(l1))-(sqrt(mean(l))))/(sqrt(mean(l1))+(sqrt(mean(l))))
    s=0
    if p>0:    
        s=0.9-(0.9 - 0.15)/(1+exp((p-0.45)/(0.05)))
    else:
        s=0.15

    l3=mean(mean(l1))+(l-mean(mean(l)))*(std1/std2)
    a3=mean(mean(a1))+(a-mean(mean(a)))*(std3/std4)
    b3=mean(mean(b1))+(b-mean(mean(b)))*(std5/std6)
    l3 = np.float64(l3)
    a3 = np.float64(a3)
    b3 = np.float64(b3)

    # Conversion from lab to RGB color space%
    L3=0.5774*l3+0.4082*a3+0.7071*b3
    M3=0.5774*l3+0.4082*a3-0.7071*b3
    S3=0.5774*l3-0.8169*a3
    L3 = np.float64(L3)
    M3 = np.float64(M3)
    S3 = np.float64(S3)
 
    R3=4.4679*L3-3.5873*M3+0.1193*S3
    G3=-1.2186*L3+2.3809*M3-0.1624*S3
    B3=0.0497*L3-0.2439*M3+1.2045*S3
 
    I7 = cv2.merge((R3,G3,B3))
    I7 = normalize(I7)
    return I7
def preProcessing0(img):
    Target = cv2.imread('/content/drive/MyDrive/20051020_45137_0100_PP.tif')
    Target = cv2.cvtColor(Target, cv2.COLOR_BGR2RGB)
    Target = cv2.resize(Target, (400, 400))
    img = ColorTransition(img, Target)
    img = normalize(img)
    return img  

def preProcessing1(img):
    # Target = cv2.imread(train_path+'/0/IDRiD_061.jpg')
    Target = cv2.imread('/content/drive/MyDrive/20051020_44782_0100_PP.tif')
    Target = cv2.cvtColor(Target, cv2.COLOR_BGR2RGB)
    Target = cv2.resize(Target, (750, 499))
    img = ColorTransition(img, Target)
    img = normalize(img)
    partA = -4 * cv2.GaussianBlur(img, (25,25), 0)
    partB = 4*img + 0.5
    result = partA + partB
    result = normalize(result)
    return result

In [ ]:
model01=keras.models.load_model("/content/drive/MyDrive/best.h5")
model12=keras.models.load_model("/content/drive/MyDrive/inc90.h5")

In [ ]:
dir1='/content/IDRID_MESSIDOR/Dataset/TrainSet'
train=os.listdir(dir1)
dir2='/content/IDRID_MESSIDOR/Dataset/TestSet'
test=os.listdir(dir2)
x_train=[]
y_train=[]
for i in train:
    a=dir1+'/'+i
    f=os.listdir(a)
    for j in f:
            if(j[0]!="."):
                x_train.append(a+"/"+j)
                y_train.append(int(i))
x_test=[]
y_test=[]
for i in test:
        a=dir2+'/'+i
        f=os.listdir(a)
        for j in f:
                if(j[0]!="."):
                    x_test.append(a+"/"+j)
                    y_test.append(int(i))

y_test_pred=[]
for i in range(len(x_test)):
        img = load_img(x_test[i],target_size=(400,400))
        img = img_to_array(img)
        img1=preProcessing0(img)
        img1 = np.expand_dims(img1, axis=0)
        a=np.argmax(model01.predict(img1))
        if(a==0):
            y_test_pred.append(a)
        else:
            img = load_img(x_test[i],target_size=(499,750))
            img = img_to_array(img)
            img2=preProcessing1(img)
            img2 = np.expand_dims(img2, axis=0)
            b=np.argmax(model12.predict(img2))+1
            y_test_pred.append(b)        
print("done")

In [ ]:
print("test:")
print(confusion_matrix(y_test, y_test_pred))
pr=precision_score(y_test, y_test_pred,average="weighted")
re=recall_score(y_test, y_test_pred,average="weighted")
print("precision:",f"{pr:.4f}")
print("recall:.",f"{re:.4f}")
print("accuracy:",f"{accuracy_score(y_test, y_test_pred):.4f}")

In [ ]:
dir1='/content/content/content/Dataset/TrainSet'
train=os.listdir(dir1)
dir2='/content/content/content/Dataset/TestSet'
test=os.listdir(dir2)
x_train=[]
y_train=[]
for i in train:
    a=dir1+'/'+i
    f=os.listdir(a)
    for j in f:
            if(j[0]!="."):
                x_train.append(a+"/"+j)
                y_train.append(int(i))
x_test=[]
y_test=[]
for i in test:
        a=dir2+'/'+i
        f=os.listdir(a)
        for j in f:
                if(j[0]!="."):
                    x_test.append(a+"/"+j)
                    y_test.append(int(i))
y_train_pred=[]
for i in range(len(x_train)):
        img = load_img(x_train[i],target_size=(400,400))
        img = img_to_array(img)
        img1=preProcessing0(img)
        img1 = np.expand_dims(img1, axis=0)
        a=np.argmax(model01.predict(img1))
        if(a==0):
            y_train_pred.append(a)
        else:
            img = load_img(x_train[i],target_size=(499,750))
            img = img_to_array(img)
            img2=preProcessing1(img)
            img2 = np.expand_dims(img2, axis=0)
            b=np.argmax(model12.predict(img2))+1
            y_train_pred.append(b)        
print("done")

done


In [ ]:
print("train:")
print(confusion_matrix(y_train, y_train_pred))
pr=precision_score(y_train, y_train_pred,average="weighted")
re=recall_score(y_train, y_train_pred,average="weighted")
print("precision:",f"{pr:.4f}")
print("recall:.",f"{re:.4f}")
print("accuracy:",f"{accuracy_score(y_train, y_train_pred):.4f}")

train:
[[1072   43   28]
 [  15   89   12]
 [   5   14  309]]
precision: 0.9363
recall:. 0.9262
accuracy: 0.9262
